In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_pickle('train_df.pickle')

In [3]:
train_df_t = (train_df[["text", "label"]])
mod_data = train_df_t.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.15 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1835/12231 as positives and unlabeling the rest


In [4]:
mod_data['class_test'] = 0
mod_data.iloc[pos_sample,-1] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 0    33730
1     1835
Name: class_test, dtype: int64


In [5]:
mod_data

,text,label,class_test
29196,he should have fucked her in the ass,1,0
64149,bitches jus b so mad at the wrong ppl i dont g...,1,0
1107,Nor I you youre putrid Youre another white kno...,1,0
23403,Go back to your country bitch,1,0
9262,@JohnCornyn @Debbie22210 The only humane thing...,1,1
...,...,...,...
61275,@purrfect1509 White people who hate white peop...,0,0
35406,I consented to the dick not the sperm,0,0
42863,STRAIGHT PEOPLE ARE CRAZY,0,0
39148,Antifa member firebombs ICE facility and is ar...,0,0


In [6]:
x_data = mod_data['text'].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [7]:
x_data

array(['he should have fucked her in the ass',
       'bitches jus b so mad at the wrong ppl i dont get it ',
       'Nor I you youre putrid Youre another white knob thinking youre virtuous while you shame people from different cultures Youre in luck Reddit loves that builds up the ol ego  if theres one thing Reddits taught me its the intolerance of the supposed tolerant Youre not new youre commonplace Dealt with racist shits like you since birth Im all gd',
       ..., 'STRAIGHT PEOPLE ARE CRAZY',
       'Antifa member firebombs ICE facility and is armed with an AR15 Wrote a manifesto using Cowfarts Cortez idea that detention facilities are CONCENTRATION CAMPS Where is the Left on condemning him CUE CRICKETS',
       'Now there is another wackadoo called Shelle that she has shifted her allegiance to While not nice at least you can through some churchy sounding words point your mother to repentance for putting some whackadoodle over the actual leadership of the church'],
      dtype=ob

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [9]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [10]:
# bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [11]:
# bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.2)(net)
net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)

model = tf.keras.Model(text_input, net)

In [13]:
opt = Adam(learning_rate = 3e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
# find the indices of the positive/labeled elements

positives = np.where(y_labeled == 1.)[0] 
print(positives)

[    4    13    23 ... 12192 12203 12211]


In [15]:
# hold_out_size = the *number* of positives/labeled samples 
# that we will use later to estimate P(s=1|y=1)
hold_out_ratio = 0.25
hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
np.random.shuffle(positives)

hold_out = positives[:hold_out_size] 
X_hold_out = x_data[hold_out]
X_hold_out[:5]

array(['@yakerriah @NomNomNyandu Fuck you you disgusting dung eating African We fucking wash Talk like this to my face you backwards cunt Fuck you Get the fuck out of Europe',
       'man fuck i hope that bitch die fuckkkk',
       'HEY COULD BE WHY TRUMP WON THEY SAW HILLARY STEAL THE ELECTION THEN ABOLISH THE CONSTITUTION THEN BRING SO MANY ARABS THAT THEY OUT NUMBERED WHITES AND THEN ALL POLITICIANS BECAME ARABS WHO THEN MADE IT ILLEGAL FOR ALL RELIGIONS AND PASSED SHARIA LAW THEN IRAN STARTED SHOOTING NUKES AND EVERY ELSE STARTED TO DO THE SAME THE WORLD WAS DESTROYED AND GOD SAID THATS IT IM NEVER MAKING WOMEN AND ARABS EVER AGAIN',
       'I am a white mans whore', 'Go cry in the shower pussy'],
      dtype=object)

In [16]:
# the actual positive *elements* that we will keep aside
 
# remove the held out elements from X and y
X = np.delete(x_data, hold_out,0) 
y = np.delete(y_labeled, hold_out)

In [17]:
y[:5]

array([0, 0, 0, 0, 1], dtype=int64)

In [18]:
X_train,X_val, y_train,y_val = train_test_split(X,y,test_size= 0.2, stratify=y)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4,restore_best_weights =True)
print(np.asarray(X_train)[:5])
model.fit(np.asarray(list(X_train)).astype('str'),np.array(list(y_train),dtype = np.float),
          validation_data =( np.array(list(X_val)),np.asarray(list(y_val), dtype = np.float) ) ,epochs = 2, batch_size =  64,callbacks = [early_stopping])
#         # We then use the estimator for prediction of the positive held-out set 
# in order to estimate P(s=1|y=1)

['Literally Jesus because he fixed America Orange man is literally satan because Mexicans'
 'Your wife was without words when she saw her Christmas present Immediately after dinner she was in the bedroom with him and you unwrapped the rest of the presents alone to the sound of them fucking Merry Christmas cucky  hotwife cuckold cuck cuckquean URL'
 'Deport them all build that wall and arrest these criminal mayors'
 'Dude seriously people like her are the most banal bored and terrified on the inside Nothing arouses this girls genuine passion she has no interests in anything beyond what her body tells her to crave and seek out You arent missing out on anything by not being with her Practice skills you love master them and wait for girls who interest you to show interest in you Seriously There are few creatures more dangerous to be involved with emotionally than a young bored girl raised in affluence Everything is a toy to them'
 'Thats an old Jewish joke']
Epoch 1/2


C:\Users\Dhwanit\anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
C:\Users\Dhwanit\anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


439/439 [==============================] - 332s 743ms/step - loss: 0.1683 - accuracy: 0.9582 - val_loss: 0.1562 - val_accuracy: 0.9608
Epoch 2/2
439/439 [==============================] - 365s 832ms/step - loss: 0.1537 - accuracy: 0.9608 - val_loss: 0.1710 - val_accuracy: 0.9608


In [19]:
hold_out_predictions = model.predict(list(X_hold_out))
hold_out_predictions

array([[0.09380719],
       [0.03868423],
       [0.0047223 ],
       [0.04936634],
       [0.02869702],
       [0.01276925],
       [0.0064201 ],
       [0.04544552],
       [0.01888394],
       [0.01439741],
       [0.07326064],
       [0.0016897 ],
       [0.02804804],
       [0.02193064],
       [0.06646208],
       [0.11490488],
       [0.015052  ],
       [0.01833756],
       [0.0168722 ],
       [0.02190655],
       [0.05363124],
       [0.03113243],
       [0.05829479],
       [0.01259108],
       [0.00421412],
       [0.04860178],
       [0.01026533],
       [0.00865922],
       [0.03786043],
       [0.00679498],
       [0.0063367 ],
       [0.01343787],
       [0.01357171],
       [0.00448719],
       [0.01561173],
       [0.05521906],
       [0.07030445],
       [0.09561294],
       [0.00393698],
       [0.02618407],
       [0.04725653],
       [0.02246998],
       [0.00794178],
       [0.09586257],
       [0.0030736 ],
       [0.01155592],
       [0.018403  ],
       [0.042

In [20]:
#take the probability that it is 1
hold_out_predictions = hold_out_predictions[:,0]
# save the mean probability 
c = np.mean(hold_out_predictions)


def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)
    predicted_s = predicted_s[:,1]
    return predicted_s / prob_s1y1

In [21]:
c

0.030370273

In [22]:
mod_data

,text,label,class_test
29196,he should have fucked her in the ass,1,0
64149,bitches jus b so mad at the wrong ppl i dont g...,1,0
1107,Nor I you youre putrid Youre another white kno...,1,0
23403,Go back to your country bitch,1,0
9262,@JohnCornyn @Debbie22210 The only humane thing...,1,1
...,...,...,...
61275,@purrfect1509 White people who hate white peop...,0,0
35406,I consented to the dick not the sperm,0,0
42863,STRAIGHT PEOPLE ARE CRAZY,0,0
39148,Antifa member firebombs ICE facility and is ar...,0,0


In [23]:
test_ds = pd.read_pickle('test_df.pickle')

In [24]:
y_pred =  model.predict(list(test_ds['text'].values))
y_pred

array([[0.00313791],
       [0.02157165],
       [0.05998213],
       ...,
       [0.00238691],
       [0.04065668],
       [0.00271142]], dtype=float32)

In [25]:
y_pred = y_pred/c

y_pred2 = [1 if i[0]>0.5 else 0 for i in y_pred]

In [26]:
from sklearn.metrics import classification_report

print(classification_report(test_ds['label'].values, y_pred2))

              precision    recall  f1-score   support

           0       0.68      0.81      0.74      2000
           1       0.77      0.62      0.69      2000

    accuracy                           0.72      4000
   macro avg       0.73      0.72      0.72      4000
weighted avg       0.73      0.72      0.72      4000



In [27]:
import os
 
pid = os.getpid()
#!kill -9 $pid